In [18]:
import pandas as pd
import xarray as xr
import numpy as np
import sys

sys.path.append('/home/users/jamd1/Isca/')
from isca_tools import cesm
from isca_tools.utils import area_weighting, lat_lon_rolling, print_ds_var_list, set_attrs
from isca_tools.papers.byrne_2021 import get_quant_ind
from isca_tools.utils.moist_physics import moist_static_energy, sphum_sat
from isca_tools.utils.constants import g
import isca_tools
import cftime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
import matplotlib.patches as patches
from tqdm.notebook import tqdm
import matplotlib
from matplotlib.dates import ConciseDateConverter
matplotlib.units.registry[cftime.DatetimeNoLeap] = ConciseDateConverter()  # to allow plotting time as x axis
# plt.style.use('/home/users/jamd1/Isca/jobs/publish.mplstyle')
import os
# from geocat.comp.interpolation import interp_hybrid_to_pressure
print(os.getcwd())

/home/users/jamd1


In [6]:
cesm.get_exp_file_dates('test', hist_file=1)

<xarray.DataArray 'time' (time: 3)> Size: 24B
array(['0041-01-02T00:00:00', '0042-01-02T00:00:00',
       '0043-01-02T00:00:00'], dtype='datetime64[s]')
Dimensions without coordinates: time

In [12]:
year_files = '41:42'
var_surf = ['TREFHT', 'QREFHT', 'PS']
ds_surf = cesm.load_dataset('test', hist_file=1, month_files=None, year_files=year_files)[var_surf]

In [13]:
ds_land = cesm.load_dataset('test', hist_file=1, month_files=None, comp='lnd', year_files=year_files)[['SOILLIQ']]
ds_land = ds_land.sum(dim='levsoi')
ds_land = ds_land.reindex_like(ds_surf['PS'], method="nearest", tolerance=0.01)

In [14]:
archive_dir = '/gws/nopw/j04/global_ex/jamd1/cesm/CESM2.1.3/archive/'
exp_name = 'test'
plev_dir = 'pressure_levels/TZ_500'
plev_path = os.path.join(archive_dir, exp_name, plev_dir)
os.listdir(plev_path)

['0041.nc', '0042.nc', '0043.nc']

In [15]:
# Each file within directory is just the year (with 4 digits) followed by .nc
year_files_all = [int(var.replace('.nc','')) for var in os.listdir(plev_path) if '.nc' in var]
# Get all years in files which are requested
year_req = isca_tools.utils.base.parse_int_list(year_files, format_func=lambda x: int(x), all_values=year_files_all)
year_req_final = [year_files_all[i] for i in range(len(year_files_all)) if year_files_all[i] in year_req]
# Load in this dataset
ds_plev = xr.open_mfdataset([os.path.join(plev_path, f'{year:04d}.nc') for year in year_req_final])

In [29]:
surf_geopotential_file = '/gws/nopw/j04/global_ex/jamd1/cesm/CESM2.1.3/cesm_inputdata/atm/cam/topo/fv_0.9x1.25_nc3000_Nsw042_Nrs008_Co060_Fi001_ZR_sgh30_24km_GRNL_c170103.nc'
ds_z2m = xr.open_dataset(surf_geopotential_file)[['PHIS']]
ds_z2m['ZREFHT'] = ds_z2m['PHIS'] / g + 2
del ds_z2m['PHIS']

ds_z2m = ds_z2m.reindex_like(ds_surf['PS'], method="nearest", tolerance=0.01)
set_attrs(ds_z2m.ZREFHT, long_name=ds_plev.Z3.long_name, units=ds_plev.Z3.units)
ds_z2m.ZREFHT

<xarray.DataArray 'ZREFHT' (lat: 5, lon: 10)> Size: 400B
array([[2.84197919e+03, 2.87674841e+03, 2.91315379e+03, 2.93361932e+03,
        2.93313907e+03, 2.91118682e+03, 2.87477533e+03, 2.84029100e+03,
        2.82628715e+03, 2.84004700e+03],
       [2.00000000e+00, 2.58515720e+00, 2.00000000e+00, 2.00000000e+00,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00, 2.00000000e+00,
        2.00000000e+00, 2.00000000e+00],
       [2.00000000e+00, 5.59191575e+02, 2.00000000e+00, 6.27186187e+01,
        2.00000000e+00, 2.00017889e+00, 2.00000000e+00, 3.45205270e+01,
        2.00000000e+00, 2.00000000e+00],
       [1.28770498e+02, 1.32412366e+02, 1.11438976e+03, 9.30985770e+02,
        2.00000000e+00, 2.00000000e+00, 9.38009648e+02, 2.30505514e+02,
        2.00000000e+00, 4.79156350e+01],
       [2.00000000e+00, 2.00000000e+00, 2.00000000e+00, 2.00000000e+00,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00, 2.00000000e+00,
        2.00000000e+00, 2.00000000e+00]])
Coordinates:
  * lat      (lat) float64 40B -90.0 -45.71 -0.4712 44.76 90.0
  * lon      (lon) float64 80B 0.0 38.75 78.75 118.8 ... 238.8 278.8 318.8 358.8
Attributes:
    long_name:  Geopotential Height (above sea level)
    units:      m

In [59]:
ds = xr.merge([ds_surf, ds_land, ds_plev, ds_z2m])

In [85]:
ds_out = xr.load_dataset('/home/users/jamd1/Isca/jobs/cesm/theory_adiabat/above_quant_2m_500hPa/test/output.nd2')